In [84]:
import pandas as pd
import os
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

os.chdir("/Users/yurizmytrakov/Desktop/")

def get_association_rules(support_threshold=0.15, lift_threshold = 0.7, confidence_threshold = 0.45):
    df = pd.read_csv('store_sales.csv')
    df = df.drop(['date_of_transaction'], axis = 1)

    orders = df.groupby('customerID')['item'].apply(list)
    orders = orders.to_frame()
    split_df = pd.DataFrame(orders['item'].tolist())

    items = set()
    for col in split_df:
        items.update(split_df[col].unique())
    items.remove(None)

    itemset = set(items)
    encoded_vals = []
    for index, row in split_df.iterrows():
        rowset = set(row) 
        labels = {}
        uncommons = list(itemset - rowset)
        commons = list(itemset.intersection(rowset))
        for uc in uncommons:
            labels[uc] = 0
        for com in commons:
            labels[com] = 1
        encoded_vals.append(labels)
    encoded_vals[0]
    ohe_df = pd.DataFrame(encoded_vals)

    freq_items = apriori(ohe_df, min_support=0.01, use_colnames=True, verbose=1)
    rules = association_rules(freq_items, metric="confidence", min_threshold=0.01)
    rules = rules[(rules['confidence']>=confidence_threshold) & (rules['support']>=support_threshold) & (rules['lift']>=lift_threshold)]
    df_rules_out = rules.sort_values('antecedent support', ascending=False)
    return df_rules_out
df_rules_out = get_association_rules()
print(df_rules_out)

Processing 18 combinations | Sampling itemset size 6
             antecedents   consequents  antecedent support  \
2033  (other vegetables)  (whole milk)            0.376603   
541         (rolls/buns)  (whole milk)            0.349666   
2193              (soda)  (whole milk)            0.313494   
2186            (yogurt)  (whole milk)            0.282966   

      consequent support   support  confidence      lift  leverage  conviction  
2033            0.458184  0.191380    0.508174  1.109106  0.018827    1.101643  
541             0.458184  0.178553    0.510638  1.114484  0.018342    1.107190  
2193            0.458184  0.151103    0.481997  1.051973  0.007465    1.045971  
2186            0.458184  0.150590    0.532185  1.161510  0.020940    1.158185  
